|Логарифмическая функция связи | $$f(x, \theta) = \exp(\sum\limits_{i=1}^n\theta_ix_i + \theta_0)$$|
|-|-|
|Функция потерь|$$l(X, y, \theta) = \frac{1}{m}\sum\limits_{i=1}^n(y_i\log\frac{y_i}{f(X_i, \theta)} - y_i + f(X_i, \theta)) + \frac{\alpha}{2}\sum\limits_{i=1}^n\theta_i^2$$|
|D2 метрика|$$D^2 = 1 - \frac{D(y, \hat{y})}{D(y, \overline{y})}$$|
|где D это:|$$D(y, \hat{y}) = 2(y\log\frac{y}{\hat{y}} - y + \hat{y})$$| 


$$\frac{\partial{l}}{\partial\theta_k} = \frac{1}{m}{\sum\limits_{i=1}^m(X_{i,k}(e^{\sum\limits_{j=1}^n\theta_jX_{i,j} + \theta_0} - y_i))+\alpha\theta_k},k=1...n$$
$$ \frac{\partial{l}}{\partial\theta_0} = \frac{1}{m}{\sum\limits_{i=1}^m(e^{\sum\limits_{j=1}^n\theta_jX_{i,j}+ \theta_0} - y_i)},k=0$$

$$X=\begin{vmatrix}
X_{1,0} & X_{1,1} & ... & X_{1,n}\\
X_{2,0} & X_{2,1} & ... & X_{2,n}\\
... & ... & ... & ...\\
X_{m,0} & X_{1,1} & ... & X_{m,n}\\
\end{vmatrix};
X_{1...m,0}=1;
\theta=\begin{vmatrix}
\theta_0 & \theta_1 & ... & \theta_n
\end{vmatrix};
y=\begin{vmatrix}
y_1 \\ y_2 \\ ... \\ y_m
\end{vmatrix};
\alpha=\begin{vmatrix}
\alpha_0 \\ \alpha_1 \\ ... \\ \alpha_n
\end{vmatrix};
\alpha_0 = 0
$$	
$$\frac{\partial{l}}{\partial\theta_k} = \frac{1}{m}{(X(e^{\theta{X}} - y_i))+{\alpha}{\theta}}$$


$$m_t = {\beta_1}{m_{t-1}} + (1-\beta_1)grad_t$$
$$v_t = {\beta_2}{v_{t-1}} + (1-\beta_2)grad_t^2$$
$$\hat{m_t}=\frac{m_t}{1-\beta_1^t}$$
$$\hat{v_t}=\frac{v_t}{1-\beta_2^t}$$
$$\theta_{t+1}=\theta_t - \frac{\eta}{\sqrt{\hat{v_t}}+\epsilon}\hat{m_t}

In [1]:
from __future__ import annotations

import numpy as np
from scipy import special

class PoissonRegression:
    
    def __init__(self, use_bias: bool = True, alpha: float = 1):
        self.use_bias = use_bias
        self.alpha = alpha
        
    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.exp(X @ self._theta + self._bias)
        
    def loss(self, X: np.ndarray, y: np.ndarray) -> float:
        y_predict = self.predict(X)
        m = len(y)
        return 1/m * (special.xlogy(y, (y / y_predict)) - y + y_predict).sum() + self.alpha/2 * (np.power(self._theta, 2).sum())

    def score(self, X: np.ndarray, y: np.ndarray) -> float:
        y_predict = self.predict(X)
        m = len(y)
        D = lambda y, y_hat: np.sum(special.xlogy(y, (y / y_hat)) - y + y_hat)
        return (1 - D(y, y_predict)/D(y, y.mean()))

    def fit(self, X: np.ndarray, y: np.ndarray, *,
            eta: float = 1e-3, beta1: float = 0.9, beta2: float = 0.999,
            epsilon: float = 1e-8, tol: float = 1e-3, max_iter: int = 1000) -> PoissonRegression:
        
        if self.use_bias:
            X_new = np.append(np.ones(shape=(X.shape[0], 1)), X , axis=1)
            cnt_example, cnt_feature = X_new.shape 
            self._coeffs    = np.zeros(cnt_feature)
            m               = np.zeros(cnt_feature)
            v               = np.zeros(cnt_feature)
            
            alpha_new = np.ones(cnt_feature)*self.alpha
            alpha_new[0] = 0  
            
            
            #Ищем оптимальные параметры ADAM'ом
            for t in range(1, max_iter+1):
                grad = find_gradient(X_new, y, cnt_example) #gradient
                if tol <= np.linalg.norm(grad):  #L2-norm
                    m = beta1 * m + (1 - beta1) * grad
                    v = beta2 * v + (1 - beta2) * grad**2
                    m_corr = m / (1 - beta1**t)
                    v_corr = v / (1 - beta2**t)
                    self._coeffs = self._coeffs - eta * m_corr / (np.sqrt(v_corr) + epsilon)
                else:
                    break
            
            self._theta = self._coeffs[1:]
            self._bias = self._coeffs[0]
        
        else:
            X_new = np.copy(X)
            cnt_example, cnt_feature = X_new.shape 
            self._coeffs    = np.zeros(cnt_feature)
            m               = np.zeros(cnt_feature)
            v               = np.zeros(cnt_feature)
              
            find_gradient = lambda X, y, m: 1/m * (np.exp(X @ self._coeffs) - y) @ X + self.alpha * self._coeffs
            
            #Ищем оптимальные параметры ADAM'ом
            for t in range(1, max_iter+1):
                grad = find_gradient(X_new, y, cnt_example) #gradient
                if tol < np.linalg.norm(grad):  #L2-norm
                    m = beta1 * m + (1 - beta1) * grad
                    v = beta2 * v + (1 - beta2) * grad**2
                    m_corr = m / (1 - beta1**t)
                    v_corr = v / (1 - beta2**t)
                    self._coeffs = self._coeffs - eta * m_corr / (np.sqrt(v_corr) + epsilon)
                else:
                    break
            
            self._theta = self._coeffs
            self._bias = 0
         
        return self

    @property
    def coeffs(self) -> np.ndarray:
        return self._theta
    
    @property
    def bias(self) -> float:
        return self._bias

In [2]:
x = np.array([[10,50],
              [20,30],
              [25,30],
              [20,60],
              [15,70],
              [40,40],
              [30,45],
              [20,45],
              [40,30],
              [7,35]])
y = np.array([1, 2, 2, 1, 1, 2, 2, 1, 2, 1])

In [3]:
clf = PoissonRegression(use_bias=True)
model = clf.fit(x, y)
model.predict(x)

array([1.0345942 , 1.56285014, 1.72699681, 1.13584479, 0.92415611,
       2.09514788, 1.62692136, 1.3323495 , 2.33031232, 1.14299596])

In [4]:
model.coeffs

array([ 0.01997456, -0.01063782])

In [5]:
model.bias

0.36615452261272596

In [6]:
model.loss(x, y)

0.021267104373499095

In [7]:
model.score(x, y)

0.7526645030513504

In [8]:
model.score(x, y)

0.7526645030513504